# IBM Data Science Project: Segmenting and Clustering Neighborhoods in Toronto

The notebook will include the following processes:

1. Scrape data related to Toronto neighborhoods and perform data pre-processing
2. Obtain location coordinates of the neighborhoods in Toronto
3. Explore and cluster neighborhoods in Toronto

## 1. Scrape data related to Toronto neighborhoods and perform data pre-processing
Import the required libraries

In [137]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Obtain the data from the html webpage

In [138]:
webpage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(webpage).text
soup = BeautifulSoup(data, 'html.parser')

In [139]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']
df_toronto = pd.DataFrame(columns=column_names)

for a in soup.find('table').find_all('tr'):
    td = a.find_all('td')
    if(len(td) > 0):
        postal_code = td[0].text.replace('\n', '')
        borough = td[1].text.replace('\n', '')
        neighborhood = td[2].text.replace('\n', '')

        df_toronto = df_toronto.append({'PostalCode': postal_code, 'Borough': borough, 'Neighborhood': neighborhood}
        , ignore_index = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Perform data pre-processing
Ignore cells with a borough that is Not assigned

In [140]:
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned'].reset_index(drop = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Since there are more than one neighborhood with the same postal code we will be combining the neighborhoods with the same postal codes seperated by a comma

In [141]:
df_toronto = df_toronto.groupby(['PostalCode', 'Borough'], as_index= False).agg(lambda x: ','.join(x))
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Replace ‘Not assigned’ neighborhoods with the same name as the borough

In [142]:
df_toronto['Neighborhood'].replace("Not assigned", df_toronto['Borough'], inplace = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [143]:
df_toronto.shape
print("The dataframe contains the following number or rows : " , df_toronto.shape[0])

The dataframe contains the following number or rows :  103


## 2. Obtain location coordinates of the neighborhoods in Toronto
Obtain the geographical coordinates of the neighborhoods using the Geocoder package csv file and merge with the existing dataframe

In [144]:
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [145]:
geo_data.rename(columns={'Postal Code': 'PostalCode'}, inplace = True)
df_toronto = pd.merge(df_toronto, geo_data, on = 'PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 3. Explore and cluster neighborhoods in Toronto
Obtain the coordinates of Toronto

In [146]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top

In [147]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

## Explore the neighborhoods using Forusquare API

In [148]:
CLIENT_ID = 'NTA5QWSTGJJIEVUXRKPKK2QNSN5T14RGDE22PKCCX001QOMZ' # your Foursquare ID
CLIENT_SECRET = 'YEJ3DJ1ACR02GFFGHY1ZUDITL5W4A5WR2BDPFB1NLB4KILAZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NTA5QWSTGJJIEVUXRKPKK2QNSN5T14RGDE22PKCCX001QOMZ
CLIENT_SECRET:YEJ3DJ1ACR02GFFGHY1ZUDITL5W4A5WR2BDPFB1NLB4KILAZ


Create a function to explore to all the neighborhoods in Toronto and obtain the nearby venues

In [149]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [150]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [151]:
print(toronto_venues.shape)
toronto_venues.head()

(2129, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Check the number of venues for each neighborhood and the number of unique categories from all the returned venues

In [152]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
...,...,...,...,...,...,...
"Willowdale, Willowdale East",33,33,33,33,33,33
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,4,4,4,4,4,4


In [153]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


### Analyze each neighborhood

In [154]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [155]:
toronto_onehot.shape

(2129, 270)

### Obtain the frequency of occurrence of each category

In [156]:
toronto_group = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_group

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.030303,0.0,0.0,0.0,0.0,0.0
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.142857,0.000000,0.0,0.0,0.0,0.0,0.0


In [157]:
toronto_group.shape

(95, 270)

### Obtain the top 5 most common venues in each neighborhoods

In [158]:
num_top_venues = 5

for hood in toronto_group['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_group[toronto_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

          venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.09
3          Restaurant  0.09
4             Butcher  0.05


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Birch Cliff, Cliffside West----
                   venue  freq
0                   Café  0.25
1        College Stadium  0.25
2  General Entertainment  0.25
3           Skating Rink  0.25
4            Yoga Studio  0.00


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2  Breakfast Spot  0.08
3          Bakery  0.08
4     Yoga Studio  0.04


----Bu

### Create a dataframe to display the top 10 venues for each neighborhood
Sort the venues in descending order

In [159]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [160]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_new = pd.DataFrame(columns=columns)
toronto_venues_new['Neighborhood'] = toronto_group['Neighborhood']

for ind in np.arange(toronto_group.shape[0]):
    toronto_venues_new.iloc[ind, 1:] = return_most_common_venues(toronto_group.iloc[ind, :], num_top_venues)

toronto_venues_new.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Skating Rink,Gym,Coffee Shop,Pharmacy,Pub,Sandwich Place,Women's Store,Distribution Center,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Bridal Shop,Sandwich Place,Diner,Restaurant,Deli / Bodega,Supermarket,Sushi Restaurant
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Restaurant,Italian Restaurant,Coffee Shop,Liquor Store,Pub,Sushi Restaurant,Café,Butcher,Pizza Place


## Cluster and analyze the Toronto neighborhoods using K-means
The neighborhoods will be segmented into 5 clusters

In [161]:
clustersize = 5

toronto_cluster = toronto_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=clustersize, random_state=0).fit(toronto_cluster)

kmeans.labels_[0:10]

array([0, 4, 0, 0, 0, 0, 0, 0, 0, 0])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [162]:
toronto_venues_new.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_cluster_final = df_toronto

toronto_cluster_final = toronto_cluster_final.join(toronto_venues_new.set_index('Neighborhood'), on='Neighborhood')

toronto_cluster_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Mexican Restaurant,Medical Center,Rental Car Location,Electronics Store,Breakfast Spot,Intersection,Bank,Discount Store,Distribution Center,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Soccer Field,Korean Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bakery,Bank,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant,Doner Restaurant,Distribution Center


### Visualize the resulting clusters generated by K-means

In [218]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(clustersize)
ys = [i + x + (i*x)**2 for i in range(clustersize)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_cluster_final['Latitude'], toronto_cluster_final['Longitude'], toronto_cluster_final['Neighborhood'], toronto_cluster_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = np.nan_to_num(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the generated clusters

The generated clusters can be analyzed and segmented based on the most common venues types

### Cluster 1

In [219]:
toronto_cluster_final.loc[toronto_cluster_final['Cluster Labels'] == 0, toronto_cluster_final.columns[[1] + list(range(5, toronto_cluster_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Construction & Landscaping,Bar,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,Scarborough,0.0,Mexican Restaurant,Medical Center,Rental Car Location,Electronics Store,Breakfast Spot,Intersection,Bank,Discount Store,Distribution Center,Dog Run
3,Scarborough,0.0,Coffee Shop,Soccer Field,Korean Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,Scarborough,0.0,Bakery,Bank,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant,Doner Restaurant,Distribution Center
6,Scarborough,0.0,Department Store,Bus Station,Discount Store,Coffee Shop,Women's Store,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Drugstore
...,...,...,...,...,...,...,...,...,...,...,...,...
87,East Toronto,0.0,Light Rail Station,Yoga Studio,Garden Center,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant
88,Etobicoke,0.0,Fast Food Restaurant,Pizza Place,Café,Fried Chicken Joint,Liquor Store,Flower Shop,Restaurant,American Restaurant,Bakery,Pharmacy
92,Etobicoke,0.0,Gym,Kids Store,Bakery,Tanning Salon,Burrito Place,Burger Joint,Sandwich Place,Supplement Shop,Discount Store,Convenience Store
95,Etobicoke,0.0,Liquor Store,Beer Store,Pet Store,Pharmacy,Pizza Place,Convenience Store,Coffee Shop,Café,Doner Restaurant,Diner


### Cluster 2

In [220]:
toronto_cluster_final.loc[toronto_cluster_final['Cluster Labels'] == 1, toronto_cluster_final.columns[[1] + list(range(5, toronto_cluster_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,1.0,Garden,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store,Dessert Shop


### Cluster 3

In [221]:
toronto_cluster_final.loc[toronto_cluster_final['Cluster Labels'] == 2, toronto_cluster_final.columns[[1] + list(range(5, toronto_cluster_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,2.0,Home Service,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dim Sum Restaurant


### Cluster 4

In [222]:
toronto_cluster_final.loc[toronto_cluster_final['Cluster Labels'] == 3, toronto_cluster_final.columns[[1] + list(range(5, toronto_cluster_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3.0,Spa,Playground,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
7,Scarborough,3.0,Bus Line,Bakery,Park,Ice Cream Shop,Metro Station,Bus Station,Intersection,Soccer Field,General Travel,Dim Sum Restaurant
14,Scarborough,3.0,Park,Playground,Sculpture Garden,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
23,North York,3.0,Park,Construction & Landscaping,Convenience Store,Bar,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,North York,3.0,Park,Food & Drink Shop,Construction & Landscaping,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
30,North York,3.0,Grocery Store,Park,Athletics & Sports,Food Truck,Liquor Store,Discount Store,Shopping Mall,Snack Place,Baseball Field,Bank
31,North York,3.0,Grocery Store,Park,Athletics & Sports,Food Truck,Liquor Store,Discount Store,Shopping Mall,Snack Place,Baseball Field,Bank
32,North York,3.0,Grocery Store,Park,Athletics & Sports,Food Truck,Liquor Store,Discount Store,Shopping Mall,Snack Place,Baseball Field,Bank
33,North York,3.0,Grocery Store,Park,Athletics & Sports,Food Truck,Liquor Store,Discount Store,Shopping Mall,Snack Place,Baseball Field,Bank
36,East York,3.0,Park,Video Store,Skating Rink,Beer Store,Curling Ice,Athletics & Sports,Pharmacy,Construction & Landscaping,Comfort Food Restaurant,Comic Shop


### Cluster 5

In [223]:
toronto_cluster_final.loc[toronto_cluster_final['Cluster Labels'] == 4, toronto_cluster_final.columns[[1] + list(range(5, toronto_cluster_final.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Dessert Shop,Fabric Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
13,Scarborough,4.0,Pharmacy,Pizza Place,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Chinese Restaurant,Thai Restaurant,Intersection,Gas Station,Convenience Store
24,North York,4.0,Grocery Store,Butcher,Bank,Pharmacy,Pizza Place,Coffee Shop,Women's Store,Doner Restaurant,Diner,Discount Store
35,East York,4.0,Pizza Place,Fast Food Restaurant,Bank,Athletics & Sports,Café,Gastropub,Intersection,Pharmacy,Gym / Fitness Center,Breakfast Spot
40,East York,4.0,Park,Pizza Place,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
81,York,4.0,Grocery Store,Bus Line,Pizza Place,Convenience Store,Women's Store,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
89,Etobicoke,4.0,Pizza Place,Skating Rink,Gym,Coffee Shop,Pharmacy,Pub,Sandwich Place,Women's Store,Distribution Center,Dim Sum Restaurant
96,North York,4.0,Pizza Place,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
98,York,4.0,Convenience Store,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
99,Etobicoke,4.0,Pizza Place,Middle Eastern Restaurant,Intersection,Discount Store,Coffee Shop,Chinese Restaurant,Sandwich Place,Distribution Center,Dim Sum Restaurant,Diner


### Cluster Segmentation

The following observations were identified upon examaning the clusters:

1. Cluster 1  consists of many resturants, coffee shops and departmental stores
2. Cluster 2 consists mostly of gardens and resturants
3. Cluster 3 is a commercial zone with mostly small businesses
4. Cluster 4 is a residential zone with many parks, trails and grocery stores
5. Cluster 5 is a combination of a residential and commercial zone with resturants, parks and various businesses